## Make time series

> Import packages

In [3]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import gridspec

# import package
import kali.carma
import numpy as np


## Assuming a single light curve parameter.

In [ ]:
# set time step
dt = 0.1

# set parameters
r_1 = (-1.0/107.8) + 0j
r_2 = (-1.0/33.2) + 0j
m_1 = (-1.0/5.5) + 0j
amp = 1.0

# translate into function input
Rho = np.require(np.array([r_1, r_2, m_1, amp]), requirements=['F', 'A', 'W', 'O', 'E'])
Theta = kali.carma.coeffs(2, 1, Rho)

#-------------------------------------------------------------------------------------------
# check if the input is valid
newTask = kali.carma.CARMATask(2, 1, nsteps = 500)
print (newTask.check(Theta))

# set input
newTask.set(dt, Theta, tnum = 0)

# simulate light curve
newLC = newTask.simulate(duration=1000.0)

# add noise, assuming 0.1% photometry
newLC.fracNoiseToSignal = 0.001
newTask.observe(newLC)

# extract light curve
plt.plot(newLC.t,newLC.y)

In [ ]:
# make many light curves
light_curve = []
for i in range(int(1e3)):
    if i % 1e2 == 0:
        print(i)
    newLC = newTask.simulate(duration=1000.0)
    light_curve.append(newLC.x)
light_curve = np.array(light_curve)
print(light_curve.shape)

np.save("light_curve.npy", light_curve)


## Vary light curve parameters.

> Make models.

In [ ]:
# set time step
dt = 0.1

# initiate generator
newTask = kali.carma.CARMATask(2, 1, nsteps = 500)
light_curve = []
t_array = []
r1_array = []
r2_array = []
m1_array = []

#-------------------------------------------------------------------------------------------
# loop over different realization
for i in range(1000):
    
    if i % 1e2 == 0:
        print(i)
        
    r_1 = np.random.uniform(50.,150.)
    r_1 = (-1.0/r_1) + 0j
    r1_array.append(r_1)
    
    r_2 = np.random.uniform(15.,45.)
    r_2 = (-1.0/r_2) + 0j
    r2_array.append(r_2)
    
    m_1 = np.random.uniform(2.5,7.5)
    m_1 = (-1.0/m_1) + 0j
    m1_array.append(m_1)
    
    amp = 1.0

#-------------------------------------------------------------------------------------------
    # translate into function input
    Rho = np.require(np.array([r_1, r_2, m_1, amp]), requirements=['F', 'A', 'W', 'O', 'E'])
    Theta = kali.carma.coeffs(2, 1, Rho)

    # check if the input is valid
    #print (newTask.check(Theta))
    
#-------------------------------------------------------------------------------------------
    # set input
    newTask.set(dt, Theta, tnum = 0)

    # simulate light curve
    newLC = newTask.simulate(duration=1000.0)
    
    # add noise, assuming 0.1% photometry
    newLC.fracNoiseToSignal = 0.001
    newTask.observe(newLC)
    
    # record light curve
    light_curve.append(newLC.y)
    t_array.append(newLC.t)
    
#-------------------------------------------------------------------------------------------
# convert the numpy array
light_curve = np.array(light_curve)
t_array = np.array(t_array)
r1_array = np.array(r1_array)
r2_array = np.array(r2_array)
m1_array = np.array(m1_array)

np.savez("light_curve.npz",\
         light_curve = light_curve,\
         t_array = t_array,\
         r1_array = r1_array,\
         r2_array = r2_array,\
         m1_array = m1_array)


0
100
200
300
400
500
600
700
800
